# Philadelphia Crime Data Project
#### William McKee
#### September 2017

In this investigation, I am analyzing Philadelphia Crime Statistics from 2006 through 2015.  The data set is available on Kaggle.  I would like to know the following about crime in my home city:

(1) What parts of the city have the most crime?

(2) What crimes are most frequently committed?

(3) Has crime improved over time?

I will look at combinations to see how crime varies geographically in Philadelphia?  Perhaps thefts and murders are committed more frequently in different parts of the city.

## Data Set Basic Statistics

The investigation begins by gathering basic data about Philadelphia crime statistics.  How many entries are in the data set?  What are the fields and ranges of each field?  What is the format of each field?  Is the formatting consistent?  What cleaning has to be performed on the data set?

In [16]:
# Initial library declarations
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Store Philadelphia's crime data
crime_data = pd.read_csv('crime.csv')

# Check out the contents
print("Data dimensions: ")
print(crime_data.shape)

Data dimensions: 
(2237605, 14)


In [17]:
# Explore column names, max values, and min values
print("Column values:")
print(crime_data.columns.values)
print("\n")

print("Column max:")
print(crime_data.max())
print("\n")

print("Column min:")
print(crime_data.min())

Column values:
['Dc_Dist' 'Psa' 'Dispatch_Date_Time' 'Dispatch_Date' 'Dispatch_Time'
 'Hour' 'Dc_Key' 'Location_Block' 'UCR_General' 'Text_General_Code'
 'Police_Districts' 'Month' 'Lon' 'Lat']


Column max:
Dc_Dist                                92
Psa                                     Z
Dispatch_Date_Time    2017-03-23 01:29:00
Dispatch_Date                  2017-03-23
Dispatch_Time                    23:59:00
Hour                                   23
Dc_Key                       201777001445
Location_Block              `732 SIGEL ST
UCR_General                          2600
Police_Districts                       22
Month                             2017-03
Lon                              -74.9575
Lat                               40.1379
dtype: object


Column min:
Dc_Dist                                       1
Psa                                           1
Dispatch_Date_Time          2006-01-01 00:00:00
Dispatch_Date                        2006-01-01
Dispatch_Time             

In [18]:
# Look at some distinct data (TODO: Pretty print)
dc_dist_distinct = crime_data.groupby('Dc_Dist')['Dc_Dist'].count()
print(dc_dist_distinct)
print("\n")

ucr_text_distinct = crime_data.groupby(['UCR_General', 'Text_General_Code']).size()
print(ucr_text_distinct)

Dc_Dist
1      48815
2     116500
3      85816
4      29198
5      31433
6      96568
7      44687
8      73820
9      84046
12    132145
14    120931
15    184677
16     73052
17     74900
18    109746
19    138987
22    127332
23     27278
24    161909
25    151245
26     86982
35    131037
39     97061
77      7813
92      1627
Name: Dc_Dist, dtype: int64


UCR_General  Text_General_Code                      
100.0        Homicide - Criminal                          3442
             Homicide - Gross Negligence                    12
             Homicide - Justifiable                         42
200.0        Rape                                        11852
300.0        Robbery Firearm                             40577
             Robbery No Firearm                          51919
400.0        Aggravated Assault Firearm                  27934
             Aggravated Assault No Firearm               68989
500.0        Burglary Non-Residential                    23276
             Burg

In [19]:
import warnings
warnings.filterwarnings("ignore", 'This pattern has match groups')

# Check dates and times for consistent formatting
def does_column_match_pattern(data_frame, column_name, pattern):
    ''' Returns true if every value of a data frame column matches a pattern
        data_frame = the pandas data frame to be checked
        column_name = name of column to be checked
        pattern = regular expression to be matched
    '''
    data = data_frame[column_name]
    return data.str.contains(pattern).all()

# Patterns copied from http://regexlib.com/DisplayPatterns.aspx?cattabindex=4&categoryId=5&AspxAutoDetectCookieSupport=1
date_time_re = '20\d{2}-((0[1-9])|(1[0-2]))-((0[1-9])|([1-2][0-9])|(3[0-1]))(\s)(([0-1][0-9])|(2[0-3])):([0-5][0-9]):([0-5][0-9])'
date_re = '20\d{2}-((0[1-9])|(1[0-2]))-((0[1-9])|([1-2][0-9])|(3[0-1]))'
time_re = '^(([0-1]?[0-9])|([2][0-3])):([0-5]?[0-9])(:([0-5]?[0-9]))?$'
print("Do all elements in columns match expected pattern?")
print("Dispatch_Date_Time: ", does_column_match_pattern(crime_data, 'Dispatch_Date_Time', date_time_re))
print("Dispatch_Date: ", does_column_match_pattern(crime_data, 'Dispatch_Date', date_re))
print("Dispatch_Time: ", does_column_match_pattern(crime_data, 'Dispatch_Time', time_re))
print("Hour: ", ((crime_data['Hour'] >= 0) & (crime_data['Hour'] <= 23)).all())
print("Month: ", does_column_match_pattern(crime_data, 'Month', '[0-9]{4}-[0-9]{2}'))



Do all elements in columns match expected pattern?
Dispatch_Date_Time:  True
Dispatch_Date:  True
Dispatch_Time:  True
Hour:  True
Month:  True


## Data Normalization

The Philadelphia Crime Data set contains over 230K rows where each row represents one 911 call about a criminal incident in the city.  Each row includes information about the date, time, police district, street, and type of incident.  There are some 2016 and 2017 incidents in the data set but there are considerable gaps in the data for those two years.  There are no apparent gaps for 2006 through 2015 so I will include only those years in the investigation.

When I looked at the distinct police districts represented in the dataset, I compared the district numbers to a map of the Philadelphia Police Districts.  There are four districts which no longer exist due to mergers.  The 4th district was merged into the 3rd district.  The 23rd was absored by the 22nd district.  The 77th was absorbed by the 12th district.  Meanwhile, the 92nd district is now part of the 14th district.  The same merger will occur in my data frame.

There are less than 1000 entries which do not contain a crime classification and corresponding description.  Since I wish to measure the crime type, I will remove these rows from the data set.

I checked all corresponding date and time columns to ensure that all values matched the expected pattern.  Below, I will check to make sure the information is consistent.  For example, if the **Dispatch_Date_Time** is '2015-11-23 19:20:00', then the **Dispatch_Date** is '2015-11-23', **Dispatch_Time** is '19:20:00', **Hour** is set to 19, and **Month** is set to '2015-11'. If there is inconsistent data in any row, the contents will be updated to match **Dispatch_Date_Time**.

The **Location_Block** field is not consistent so there will be some cleaning needed to ensure the same pattern in the data.

##### TODO: Investigate this further and look at ways to clean this field.
##### TODO: More than one 911 call for same incident; what are criteria for two rows to denote same event?

In [21]:
from datetime import datetime
from datetime import timedelta

# Assure consistent dates and times in each row
def ensure_data_time_consistency(data_frame):
    ''' Checks all date/time fields to be sure there is a match
        Updates the date/time fields if not a match!
        data_frame = data frame to investigate
    '''
    # Loop over each row
    for tup in data_frame.itertuples():
        # Record index
        this_index = tup.Index
        
        # Get Dispatch_Date_Time
        full_date_time = datetime.strptime(tup.Dispatch_Date_Time, "%Y-%m-%d %H:%M:%S")
        
        # Check Dispatch_Date
        date_only = datetime.strptime(tup.Dispatch_Date, "%Y-%m-%d")
        if (date_only.date() - full_date_time.date() > timedelta(seconds = 59)):
            data_frame.loc[this_index, 'Dispatch_Date'] = str(full_date_time.year) + '-' + \
                                                          str(full_date_time.month) + '-' + \
                                                          str(full_date_time.day)
        
        # Check Dispatch Time
        time_only = datetime.strptime(tup.Dispatch_Time, "%H:%M:%S")
        if (time_only.time() != full_date_time.time()):
            data_frame.loc[this_index, 'Dispatch_Time'] = str(full_date_time.hour) + '-' + \
                                                          str(full_date_time.minute) + '-' + \
                                                          str(full_date_time.second)
        
        # Check Hour
        if (tup.Hour != full_date_time.hour):
            data_frame.loc[this_index, 'Hour'] = full_date_time.hour
        
        # Check Month
        month_only = datetime.strptime(tup.Month, "%Y-%m")
        if (month_only.year != full_date_time.year or month_only.month != full_date_time.month):
            data_frame.loc[this_index, 'Month'] = str(full_date_time.year) + '-' + str(full_date_time.month)
    
# Ensure the dates and times match in each row
ensure_data_time_consistency(crime_data)